# Technical task code

## Loading in the data

Importing libraries

In [11]:
%matplotlib inline
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree
from scipy import stats
import statistics
import pvlib
from datetime import datetime
import seaborn as sns

Getting US 2012 wind data from NREL AWS servers

In [12]:
f = h5pyd.File("/nrel/wtk/conus/wtk_conus_2012.h5", 'r')

Subsetting to just wind speed at an altitude of 10 meters

In [13]:
dset = f['windspeed_10m']

Extracting date and time index

In [ ]:
time_index = pd.to_datetime(f['time_index'][...].astype(str))
time_index # Temporal resolution is 1 hr

Windspeeds need to be scaled by 100 to be in native value of meters per second

In [ ]:
scale_factor = dset.attrs['scale_factor']
dset[0,0]/scale_factor

Subsetting the 10m altitude wind speed data to a single location in New York City and converting to a dataframe.

In [ ]:
dset_coords = f['coordinates'][...]
tree = cKDTree(dset_coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

NewYorkCity = (40.7128, -74.0059)
NewYorkCity_idx = nearest_site(tree, NewYorkCity[0], NewYorkCity[1] )

print("Site index for New York City: \t\t {}".format(NewYorkCity_idx))
print("Coordinates of New York City: \t {}".format(NewYorkCity))
print("Coordinates of nearest point: \t {}".format(dset_coords[NewYorkCity_idx]))

tseries = dset[:, NewYorkCity_idx] / dset.attrs['scale_factor']
df = pd.DataFrame({'wspd': tseries}, index=time_index)
df["year"] = df.index.year
df["month"] = df.index.month
df["day"] = df.index.day
df["hour"] = df.index.hour

df.head()

## Monthly wind speed variability

The following plot shows the variation in windspeed across the months in New York City.  Based upon the monthy speed distributions, the fall and winter months seem to have the most variability.

In [ ]:
speedByMonth = [df.loc[df['month'] == month]['wspd'].values for month in np.arange(1, 13)]

plt.boxplot(speedByMonth, boxprops={'zorder': 2})
plt.title("Monthly variation in wind speed in NYC, 2012")
plt.ylabel("Wind speed (m/s)")
plt.xlabel("Month")
plt.xticks(np.arange(1, 13, 1),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])


plt.violinplot(speedByMonth)
plt.title("Monthly variation in wind speed in NYC, 2012")
plt.ylabel("Wind speed (m/s)")
plt.xlabel("Month")
plt.xticks(np.arange(1, 13, 1),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.show()



### Statistical measures of variability

Lee et al. (2018) recommend using the robust coefficient of variation (RCoV) to assess wind speed variability.  Despite Lee et al. (2018) suggesting that at least 10 years of records are needed to acheive high confidence , RCoV converged to the study period value relatively quickly, so it is still a useful metric for this purpose.  I also present Kurtosis and Skewness as they coverged in a single year.

In [ ]:
def rcov(vals):
    return stats.median_abs_deviation(vals)/statistics.median(vals)

In order to maximise scalability, here is a function that takes in a set of coordinates, finds the closest 2x2km grid cell and calculates monthly RCoV for 2012.

In [ ]:
dset_coords = f['coordinates'][...]
tree = cKDTree(dset_coords)

def getMonthlyRCoV2012(lat, lon):
    idx = nearest_site(tree, lat, lon)
    tseries = dset[:, idx] / dset.attrs['scale_factor']
    df = pd.DataFrame({'wspd': tseries}, index=time_index)
    df["year"] = df.index.year
    df["month"] = df.index.month
    
    speedByMonth = [df.loc[df['month'] == month]['wspd'].values for month in np.arange(1, 13)]
    
    monthlyRCoVs = [rcov(speedByMonth[month]) for month in np.arange(0, 12, 1)]
    d = {'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        'RCoV': monthlyRCoVs}
    
    return(pd.DataFrame(data=d).set_index("month"))

In [ ]:
variability = getMonthlyRCoV2012(40.7128, -74.0059)
variability

In [ ]:
speedByMonth = [df.loc[df['month'] == month]['wspd'].values for month in np.arange(1, 13)]

#calculating monthly skewness
monthlySkew = [stats.skew(speedByMonth[month]) for month in np.arange(0, 12, 1)]

#calculating monthly kurtosis
monthlyKurt = [stats.kurtosis(speedByMonth[month]) for month in np.arange(0, 12, 1)]

#adding to table
variability['skewness'] = monthlySkew
variability['kurtosis'] = monthlyKurt
variability


All the metrics indicate high wind speed variability in October.  Interestingly, the metrics differ for September: both Skewness and Kurtosis have high measures while RCoV has a relatively low measure.  This is likely because of the relatively clumped distribution of the outlying windspeed records.

In [ ]:
plt.plot(variability['RCoV'])
plt.title('Wind speed variability metrics by month, NYC 2012')
plt.ylabel('RCov')
plt.show()
plt.plot(variability['skewness'])
plt.ylabel('Skewness')
plt.show()
plt.plot(variability['kurtosis'])
plt.ylabel('Kurtosis')
plt.xlabel('Month')
plt.show()

### Combined hourly and monthly variability

The following plots show the average hourly wind speeds and RCoVs for 2012 in New York City.  It regularly seems to be windier at night, with summer months have slower winds especially during the days.  There seems to be generally lower variability at night.

In [ ]:
aggSpeed = df.groupby(["month","hour"]).mean()
aggSpeed = aggSpeed.reset_index().pivot(index="month",columns="hour",values="wspd")

plt.imshow(aggSpeed)
plt.xlabel("Hour")
plt.ylabel("Month")
plt.yticks(np.arange(0, 12, 1),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.title("Mean Hourly Wind Speed (m/s) \n in New York City (2012)")
plt.colorbar()
plt.show()

aggRCov = df[['wspd', 'month', 'hour']]
aggRCov = aggRCov.groupby(["month","hour"]).agg(lambda x: rcov(x)).rename(columns = {'wspd':'rcov'})
aggRCov = aggRCov.reset_index().pivot(index="month",columns="hour",values="rcov")

plt.imshow(aggRCov, cmap = 'plasma')
plt.xlabel("Hour")
plt.ylabel("Month")
plt.yticks(np.arange(0, 12, 1),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.title("Hourly Wind Speed RCoV \n in New York City (2012)")
plt.colorbar()
plt.show()



In [ ]:
#add plot here scatterplot of mean monthly wind (Y) speeds and monthly RCoVs (x) to show best months to have generate power

## Diurnal variability

The first step is to differentiate day and night times based off of our site location and time.

In [ ]:
#reading in sunrise and sunset data
sunriseSunset = pvlib.solarposition.sun_rise_set_transit_ephem(df.index.tz_localize('EST') , 40.7128, -74.0059)

#data is hourly, so only need the hour from above table
df['sunrise'] = sunriseSunset['sunrise'].dt.hour.values
#without +1 a sunset at 4:14pm would mark timestamp of 4 pm as night, even though it is before 4:14pm
df['sunset'] = sunriseSunset['sunset'].dt.hour.values + 1 
afterSunrise = df['hour'] > df['sunrise']
beforeSunset = df['hour'] < df['sunset']

#daytime accounts for the changes in time across the year
df['daytime'] = afterSunrise.values & beforeSunset.values
df.head(10)

The following plot shows the variation in day and night wind speeds across the months of 2012.  There does not seem to be strong difference between day and night average wind speeds. 

In [ ]:
day = df[df['daytime'] == True][['wspd', 'month']].reset_index(drop = True)
day['Time'] = 'day'
night = df[df['daytime'] == False][['wspd', 'month']].reset_index(drop = True)
night['Time'] = 'night'

dayNight = pd.concat([day, night])

sns.boxplot(x='month',y='wspd',data=dayNight,hue='Time')

plt.xticks(np.arange(0, 12, 1),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylabel("Wind speed (m/s)")
plt.xlabel("")
#add title

plt.show()


However, when looking at variability in wind speed during the day and night, we see a slightly different picture. Summer and fall months (except August) seem to have the greatest differences in variability.

In [ ]:
#adapting function to calculate RCoV for day and night separately

dset_coords = f['coordinates'][...]
tree = cKDTree(dset_coords)

def getMonthlyRCoV2012dayNight(lat, lon, tz):
    idx = nearest_site(tree, lat, lon)
    tseries = dset[:, idx] / dset.attrs['scale_factor']
    df = pd.DataFrame({'wspd': tseries}, index=time_index)
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["hour"] = df.index.hour

    #reading in sunrise and sunset data
    sunriseSunset = pvlib.solarposition.sun_rise_set_transit_ephem(df.index.tz_localize(tz) , lat, lon)

    #data is hourly, so only need the hour from above table
    df['sunrise'] = sunriseSunset['sunrise'].dt.hour.values
    #without +1 a sunset at 4:14pm would mark timestamp of 4 pm as night, even though it is before 4:14pm
    df['sunset'] = sunriseSunset['sunset'].dt.hour.values + 1 
    afterSunrise = df['hour'] > df['sunrise']
    beforeSunset = df['hour'] < df['sunset']

    #daytime accounts for the changes in time across the year
    df['daytime'] = afterSunrise.values & beforeSunset.values

    #splitting into day and night
    day = df[df['daytime'] == True]
    night = df[df['daytime'] == False]
    speedByMonthDay = [day.loc[day['month'] == month]['wspd'].values for month in np.arange(1, 13)]
    speedByMonthNight = [night.loc[night['month'] == month]['wspd'].values for month in np.arange(1, 13)]

    #calculating RCoV for day and night separately
    daytimeMonthlyRCoVs = [rcov(speedByMonthDay[month]) for month in np.arange(0, 12, 1)]
    nightMonthlyRCoVs = [rcov(speedByMonthNight[month]) for month in np.arange(0, 12, 1)]
    d = {'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            'dayRCoV': daytimeMonthlyRCoVs,
            'nightRCoV': nightMonthlyRCoVs}
    return(pd.DataFrame(data=d).set_index("month"))

In [ ]:
getMonthlyRCoV2012dayNight(40.7128, -74.0059, 'EST')

In [ ]:
fig, ax = plt.subplots()
lines = ax.plot(getMonthlyRCoV2012dayNight(40.7128, -74.0059, 'EST'))
ax.legend(['day', 'night'])
plt.ylabel('RCoV')
plt.show()
